In [1]:
! pip install pandas matplotlib torch cairosvg tqdm openai

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
import cairosvg
import numpy as np
import io
import math
import sys
import base64
import json
import time
import csv
from tqdm import tqdm
from openai import OpenAI

# ---- CONFIGURATION ----
MODEL_NAME = "gpt-4o-mini"   # Vision-language model
# Initializing OpenAI client - see https://platform.openai.com/docs/quickstart?context=python
OPENAI_KEY = "<OPENAI_KEY>"
client = OpenAI(api_key=OPENAI_KEY)

CUSTOM_PROMPT = """
You are an AI Agent with specilaised knowledge in reading and understanding map data. Analyze the following map and using information from the steps and examples given below, answer the question.

Steps to follow:

Identify Map-Related Elements in the Question
 Carefully understand the question to determine the key geographical features, locations, or spatial characteristics it refers to.


Locate the Identified Elements on the Map
 Find and observe these features or entities on the map provided. Pay attention to patterns, distributions, directions, scales, or any visual cues.


Apply Logical Reasoning
 Use spatial reasoning and contextual clues from the map to draw connections between the question and the observed map features.


Formulate a Concise Answer
 Based on your reasoning, arrive at a clear and accurate answer. Return only a word or phrase, as required—no explanation is needed. 
 If adequate data is not present, give answer as "no data". 
 If you have all the data and there is no answer to the question, give answer "none". If it is a counting problem, give answer 0. 
 If you have all the data and it is not possible to answer the question, give answer "not possible".

Assuming we are talking about a map with election results for USA. This  map contains  the voter breakdown across the United States, including the number of votes cast and the winning party in each state. Some examples of questions and there answers are as follows:


Question: Count the number states on the west coast where Democrats won.
Answer: 3


Questions: Based on the information given in the map, who won the election, Democrats or Republicans?
Answer: Democrats

Questions: Based on the information given in the map, if both Democrats and Republicans win 25 states each, do we have more blue states or red states?
Answer: neither

Question: List the top 4 states in terms of seats where the republicans won
Answer: Texas, Georgia, Missouri, Tenessee

Question: Rank these states in ascending order of seats - kansas, south carolina, nebraska, oklahoma, colorado, wisconsin
Answer: nebraska, kansas, oklahoma, south carolina, colorado, wisconsin

Question: Based on reasoning, Answer the following:
     Montana : Wyoming :: North Dakota : ?
Answer: South Dakota

Now, Answer the Question below based on the information, instruction and examples above:

"""
INPUT_CSV = "data/typed_questions.csv"
OUTPUT_CSV = f"results/{MODEL_NAME.split('/')[-1]}_results.csv"  
IMAGE_BASE_FOLDER = "data/imgs/"

# ---- HELPER FUNCTIONS ----
def find_image_recursive(base_folder: str, image_name: str) -> str | None:
    """Recursively find image by filename in base_folder. Returns full path or None."""
    for root, _, files in os.walk(base_folder):
        if image_name in files:
            return os.path.join(root, image_name)
    return None


def load_image_safely(image_path: str, output_format: str = "PNG"):
    """
    Load an image safely and return a base64-encoded data URI.
    Supports PNG, JPEG, GIF, WEBP, and SVG.
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    except Exception as e:
        print(f"❌ Failed to load image {image_path}: {e}")
        return None

def ask_image_question(image_path: str, question: str, custom_prompt: str, index:int) -> str:
    image = load_image_safely(image_path)
    if image is None:
        return "Error loading image"
    
    # ✅ OpenAI requires chat template
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": MODEL_NAME,
            "temperature": 0.0,
            "max_tokens": 128,
            "messages": [
                {
                    "role": "system",
                    "content": custom_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": question
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image}"
                            }
                        },
                    ],
                }
            ]            
        }
    }
    
    return task

# Load already answered pairs from output CSV to a set for quick lookup
def load_already_answered(output_csv):
    answered_set = set()
    try:
        df_out = pd.read_csv(output_csv)
        for _, row in df_out.iterrows():
            answered_set.add( (row['image_name'], row['question']) )
    except FileNotFoundError:
        # Output file does not exist yet
        pass
    return answered_set

In [6]:
output_csv = OUTPUT_CSV

In [7]:
files = [
    "file-Bf146pnSQAAYWUtRYJZfen",
    "file-LRNt3j55V96mk64QfsGMVd",
    "file-SJp4p9wxwzjrnBBxkJdE6M",
    "file-CeCzQLtcuue6m9SSAUHPvo"
]

In [ ]:
for i in range(len(files)):

    batch_job = client.batches.retrieve(files[i])
    status = batch_job.status
    print(f"⏳ Batch status: {status}")

    if batch_job.status != "completed":
        print(f"❌ Batch did not complete successfully: {batch_job.status}")
        continue

    # Save all result files (guaranteed output)
    result_file_name = "openai_results/batch_job_results_mapqa.json"
    with open(result_file_name, "w") as out_file:
        print(batch_job)
        json.dump(json.loads(batch_job.model_dump_json()), out_file, indent=4)
        # Replace with your actual file ID from the batch status
        output_file_id = batch_job.output_file_id

        # Download the file content
        output_file = client.files.content(output_file_id)

        # The content is a JSONL (JSON Lines) file, one JSON per line
        lines = output_file.text.splitlines()

        results = []
        for line in lines:
            data = json.loads(line)

            if "response" not in data or "body" not in data["response"]:
                continue

            try:
                content = data["response"]["body"]["choices"][0]["message"]["content"]
                if isinstance(content, list):  # new format
                    text = "".join([c.get("text", "") for c in content if c.get("type") == "text"])
                else:  # fallback if it’s already a string
                    text = content
                results.append(text)
            except Exception as e:
                print("Skipping malformed line:", e)

        print("Total results:", len(results))

    with open(output_csv, mode="a", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["image_name", "question", "llm_answer"])
        
        # Write header only if file was just created (empty)
        if f_out.tell() == 0:
            writer.writeheader()

        for (idx, row), answer in zip(df.iterrows(), results):
            image_name = row["image_name"]
            question = row["question"]

            llm_answer = answer if answer else "❌ No answer returned"

            writer.writerow({
                "image_name": image_name,
                "question": question,
                "llm_answer": llm_answer
            })

    print(f"✅ Results written to {output_csv}")

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'batch_id': 'file-Bf146pnSQAAYWUtRYJZfen'. Expected an ID that begins with 'batch'.", 'type': 'invalid_request_error', 'param': 'batch_id', 'code': 'invalid_value'}}

In [10]:
import json
import csv

# Your result file IDs (from completed batch jobs)
file_ids = [
    "file-Bf146pnSQAAYWUtRYJZfen",
    "file-LRNt3j55V96mk64QfsGMVd",
    "file-SJp4p9wxwzjrnBBxkJdE6M",
    "file-CeCzQLtcuue6m9SSAUHPvo"
]

output_csv = OUTPUT_CSV

for file_id in file_ids:
    print(f"📂 Processing result file: {file_id}")

    # Download the file content
    output_file = client.files.content(file_id)

    # The content is a JSONL (JSON Lines) file, one JSON per line
    lines = output_file.text.splitlines()

    results = []
    for line in lines:
        data = json.loads(line)

        if "response" not in data or "body" not in data["response"]:
            continue

        try:
            content = data["response"]["body"]["choices"][0]["message"]["content"]
            if isinstance(content, list):  # new format
                text = "".join([c.get("text", "") for c in content if c.get("type") == "text"])
            else:  # fallback if it’s already a string
                text = content
            results.append(text)
        except Exception as e:
            print("⚠️ Skipping malformed line:", e)

    print(f"✅ Extracted {len(results)} answers from {file_id}")

    # Save results alongside your dataframe rows
    with open(output_csv, mode="a", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["image_name", "question", "llm_answer"])

        # Write header only once
        if f_out.tell() == 0:
            writer.writeheader()

        for (idx, row), answer in zip(df.iterrows(), results):
            image_name = row["image_name"]
            question = row["question"]

            llm_answer = answer if answer else "❌ No answer returned"

            writer.writerow({
                "image_name": image_name,
                "question": question,
                "llm_answer": llm_answer
            })

print(f"🎉 All results written to {output_csv}")


📂 Processing result file: file-Bf146pnSQAAYWUtRYJZfen
✅ Extracted 0 answers from file-Bf146pnSQAAYWUtRYJZfen


NameError: name 'df' is not defined